In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import linear_kernel
import json

In [68]:
class Bartender():
    def __init__(self, file_path="../dataset/cocktail_all.csv"):
        self.df = pd.read_csv(file_path)
        self.vectorizer = None
        self.tfidf_matrix = None
        self.feature_df = None
        self.similarity_df = None

        self.name_vectorizer = None
        self.name_tfidf_matrix = None
        self.name_df = None
        self.name_similarity_df = None

        self.image_url = None

    def load_analizer(self):
        self.df.fillna('', inplace=True)
        stop_words = frozenset(['oz', 'simple', 'dash', 'bsp', 'drops', "ml", "grams", "gram"])
        self.vectorizer = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS.union(stop_words), token_pattern=r'\b[^\d\W][^\d\W]+\b')
        self.tfidf_matrix = self.vectorizer.fit_transform(self.df['Ingredients'])
        self.feature_df = pd.DataFrame(self.tfidf_matrix.toarray(), columns=self.vectorizer.get_feature_names(), index=self.df['Cocktail Name'])
        similarity_matrix = linear_kernel(self.tfidf_matrix, self.tfidf_matrix)
        self.similarity_df = pd.DataFrame(similarity_matrix, columns=self.feature_df.index, index=self.feature_df.index)

        self.name_vectorizer = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)
        self.name_tfidf_matrix = self.name_vectorizer.fit_transform(self.df['Cocktail Name'])
        self.name_df = pd.DataFrame(self.name_tfidf_matrix.toarray(), columns=self.name_vectorizer.get_feature_names(), index=self.df['Cocktail Name'])
        name_similarity_matrix = linear_kernel(self.name_tfidf_matrix, self.name_tfidf_matrix)
        self.name_similarity_df = pd.DataFrame(name_similarity_matrix, columns=self.name_df.index, index=self.name_df.index)

        self.image_url = pd.read_csv("../dataset/image_url.csv")

    def convert_df_2_json(self, df, names):
        idx = []
        for i, name in enumerate(list(df["Cocktail Name"])):
            idx.append(np.argwhere(np.array(names) == name)[0][0])
        idx = np.array(idx)
        out = {"name":[], "all_ingredients": [], "preparation": [], "image": []}
        out["name"] = list(np.array(list(df["Cocktail Name"]))[idx])
        out["all_ingredients"] = list(np.array(list(df["All Ingredients"]))[idx])
        out["preparation"] = list(np.array(list(df["Preparation"]))[idx])
        out["image"] = list(np.array(self.get_image_url_list(out["name"]))[idx])
        return json.dumps(out)
            

    def get_similarity_matrix_by_ingredients(self, user_input):
        user_input_vector = self.vectorizer.transform([user_input])
        similarity_vector = linear_kernel(self.tfidf_matrix, user_input_vector)
        return pd.DataFrame(similarity_vector, columns=['Similarity'], index=self.df['Cocktail Name']).sort_values(by='Similarity', ascending=False)

    def get_recommendation_list_by_ingredient(self, user_input, k=10):
        similar_items = self.get_similarity_matrix_by_ingredients(user_input)
        return self.convert_df_2_json(self.df[self.df['Cocktail Name'].isin(similar_items[:k].index)], list(similar_items[:k].index))

    def get_similarity_matrix_by_name(self, user_input, k=5):
        user_input_vector = self.name_vectorizer.transform([user_input])
        similarity_vector = linear_kernel(self.name_tfidf_matrix, user_input_vector)
        return pd.DataFrame(similarity_vector, columns=['Similarity'], index=self.df['Cocktail Name']).sort_values(by='Similarity', ascending=False)

    def get_recommendation_list_by_name(self, user_input, k=5):
        similar_items = self.get_similarity_matrix_by_name(user_input)
        return self.convert_df_2_json(self.df[self.df['Cocktail Name'].isin(similar_items[:k].index)], list(similar_items[:k].index))

    def get_recommendation_list(self, user_input, k=10):
        similar_items_by_ingredient = self.get_similarity_matrix_by_ingredients(user_input)
        similar_items_by_name = self.get_similarity_matrix_by_name(user_input)

        similar_items = pd.concat([similar_items_by_ingredient, similar_items_by_name], axis=0)
        similar_items_sorted = similar_items.sort_values(by='Similarity', ascending=False)[:k]
        return self.convert_df_2_json(self.df[self.df['Cocktail Name'].isin(similar_items_sorted[:k].index)], list(similar_items_sorted[:k].index))

    def get_image_url_list(self, names, placeHolder_addrss="http://www.thecocktaildb.com/images/media/drink/wpxpvu1439905379.jpg"):
        url_list = []
        for name in names:
            if name in np.array(self.image_url["Name"]):
                url_list.append(np.array(self.image_url.loc[self.image_url["Name"] == name, "url"])[0])
            else:
                rand_idx = int(np.random.rand()*len(self.image_url["Name"]))
                rand_name = self.image_url["Name"][rand_idx]
                placeHolder_addrss = np.array(self.image_url.loc[self.image_url["Name"] == rand_name, "url"])[0]
                url_list.append(placeHolder_addrss)
        return url_list



In [69]:
bartender = Bartender(file_path="/Users/xin/Next_Era_Elite_Bartender/dataset/cocktail_all.csv")
bartender.load_analizer()

Search by ingredients

In [70]:
bartender.get_recommendation_list_by_ingredient("lemon", k=5)

'{"name": ["RUM COOLER", "69 SPECIAL", "WHISKEY SOUR", "ZIZI COIN-COIN", "TEQUILA SOUR"], "all_ingredients": ["Lemon, Lemon-lime soda, Rum, ,", "Lemon juice, 7-Up, Gin, ,", "Lemon, Cherry, Powdered sugar, Lemon, Blended whiskey, ,", "Lemon, Ice, Lemon juice, Cointreau, ,", "Cherry, Lemon, Powdered sugar, Lemon, Tequila, ,"], "preparation": ["Pour the rum and soda into a collins glass almost filled with ice cubes. Stir well and garnish with the lemon wedge.", "Pour 2 oz. gin. Add 4 oz. 7-up. Add Lemon Juice for flavor. If you are weak, top up glass with more 7-Up.", "Shake with ice. Strain into chilled glass, garnish and serve. If served \'On the rocks\', strain ingredients into old-fashioned glass filled with ice.", "Pour 5cl of Cointreau on ice, add 2cl of fresh lemon (or lime) juice, stir gently, and finally add slices of lemon/lime in glass.", "Shake tequila, juice of lemon, and powdered sugar with ice and strain into a whiskey sour glass. Add the half-slice of lemon, top with the c

Search by Name

In [71]:
bartender.get_recommendation_list_by_name("long island", k=5)

'{"name": ["LONG ISLAND TEA", "RADIOACTIVE LONG ISLAND ICED TEA", "LONG ISLAND LIMONCELLO MARTINI", "3-MILE LONG ISLAND ICED TEA", "LONG ISLAND ICED TEA"], "all_ingredients": ["Coca-Cola, Lemon, Tequila, Gin, Light rum, Vodka, ,", "Malibu rum, Midori melon liqueur, Chambord raspberry liqueur, Triple sec, Gin, Tequila, Vodka, Rum, ,", "1 oz Hangar 1 Buddha\'s Hand Citron Vodka, 1 oz Luxardo Limoncello, .75 oz Fresh Lemon Juice, .25 oz Monin Cane Syrup,Lemon Wheel", "Lemon, Bitters, Sweet and sour, Coca-Cola, Vodka, Triple sec, Tequila, Light rum, Gin, ,", "Lemon peel, Coca-Cola, Gin, Light rum, Tequila, Vodka, ,"], "preparation": ["Combine all ingredients (except cola) and pour over ice in a highball glass. Add the splash of cola for color. Decorate with a slice of lemon and serve.", "Pour all ingredients over ice in a very tall glass. Sip cautiously.", "Served up and finished with a lemon wheel.", "Fill 14oz glass with ice and alcohol. Fill 2/3 glass with cola and remainder with sweet 

Search by ingredient/name

In [62]:
bartender.get_recommendation_list("155, sugar, lemon, cola", k=10)

,Unnamed: 0,Cocktail Name,Ingredients,Garnish,Preparation,All Ingredients
90,90,1880'S AMERICANA,"2 oz Hirsch Small Batch Reserve Bourbon, 10 ml...",Cola candied orange peel,Build over ice in Old Fashioned glass.,"2 oz Hirsch Small Batch Reserve Bourbon, 10 ml..."
189,190,PINK & BLACK,"2 oz Pink Pigeon Rum, 3 oz Coca-Cola, .5 oz Li...",,"Combine the Pink Pigeon, the cola, and the lim...","2 oz Pink Pigeon Rum, 3 oz Coca-Cola, .5 oz Li..."
684,0,155 BELMONT,"Orange juice, Vodka, Light rum, Dark rum,",,Blend with ice. Serve in a wine glass. Garnish...,"Orange juice, Vodka, Light rum, Dark rum, ,"
692,8,410 GONE,"Coca-Cola, Peach Vodka,",,,"Coca-Cola, Peach Vodka, ,"
747,65,CUBA LIBRE,"Coca-Cola, Lime, Light rum,",,Build all ingredients in a Collins glass fille...,"Coca-Cola, Lime, Light rum, ,"
867,188,KIWI LEMON,"Ice, Bitter lemon, Kiwi liqueur,",,Mix in highball glass. Stirr. Garnish with sli...,"Ice, Bitter lemon, Kiwi liqueur, ,"
876,197,LEMON SHOT,"151 proof rum, Sugar, Lemon, Absolut Citron, G...",,Mix Galliano and Absolut Citron in a shot glas...,"151 proof rum, Sugar, Lemon, Absolut Citron, G..."
878,199,LEMON DROP,"Lemon, Cointreau, Absolut Vodka,",,Shake and strain into a chilled cocktail glass...,"Lemon, Cointreau, Absolut Vodka, ,"
884,205,LONG ISLAND TEA,"Coca-Cola, Lemon, Tequila, Gin, Light rum, Vod...",,Combine all ingredients (except cola) and pour...,"Coca-Cola, Lemon, Tequila, Gin, Light rum, Vod..."
888,209,LONG ISLAND ICED TEA,"Lemon peel, Coca-Cola, Gin, Light rum, Tequila...",,Mix all contents in a highball glass and sitr ...,"Lemon peel, Coca-Cola, Gin, Light rum, Tequila..."


In [64]:
bartender.get_image_url_list(["ZOKSEL", "LEMON SHOT", "LONG ISLAND TEA", "none"])

['http://www.thecocktaildb.com/images/media/drink/ft8ed01485620930.jpg',
 'http://www.thecocktaildb.com/images/media/drink/mx31hv1487602979.jpg',
 'http://www.thecocktaildb.com/images/media/drink/ywxwqs1439906072.jpg',
 'path/to/local/address']

In [67]:
idx = np.array([2,1,0])
np.array([1,2,3])[idx]

array([3, 2, 1])